# Tiny Anime GPT 

This notebook trains a Tiny GPT model from scratch on the **Japanese Anime IMDb Dataset**.

### Instructions
1. **Runtime:** Ensure you are connected to a GPU Runtime (Runtime > Change runtime type > T4 GPU).
2. **Data:** Upload your Kaggle CSV file (e.g., `anime.csv`) to the **Files** sidebar on the left.
3. **Goal:** The model will learn to generate plausible Anime Titles, Genres, and Synopses.

## 1. Setup & Hardware Check

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os

# Check for GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Check GPU details if available
if device == 'cuda':
    !nvidia-smi

## 2. Data Preparation (CSV to Text)
This section finds your uploaded CSV and converts it into a plain text stream for the model.

In [ ]:
# --- Load Anime Data ---
# Find any CSV file in the current directory
csv_files = glob.glob("*.csv")

if not csv_files:
    print("\u26A0\uFE0F NO CSV FOUND! Please upload your anime.csv file to the sidebar.")
    # Fallback to dummy text so code doesn't crash
    text = "Title: Naruto \n Genre: Action \n Synopsis: Ninjas fighting. " * 1000
else:
    file_path = csv_files[0]
    print(f"Loading data from: {file_path}")
    
    try:
        df = pd.read_csv(file_path)
        print("Columns found:", df.columns.tolist())
        
        # --- Preprocessing Strategy ---
        # We combine columns into a structured text block.
        # Adjust these column names if your CSV is slightly different.
        # Common Kaggle column names: 'Title', 'Genre', 'Description', 'Plot', 'Synopsis'
        
        text_corpus = []
        
        # Iterate over rows and create a string for each anime
        for index, row in df.iterrows():
            # Use .get() to avoid errors if column names differ slightly
            title = str(row.get('Title', row.get('title', 'Unknown Title')))
            genres = str(row.get('Genre', row.get('genres', 'Unknown')))
            # Clean up plot/description
            plot = str(row.get('Description', row.get('Plot', row.get('synopsis', ''))))
            
            # Create a formatted string. 
            # We use distinct markers so the model learns the structure.
            entry = f"ANIME: {title}\nGENRES: {genres}\nSYNOPSIS: {plot}\n<END>\n\n"
            text_corpus.append(entry)
            
        text = "".join(text_corpus)
        print(f"\nData loaded successfully! Total Corpus Length: {len(text)} characters")
        print(f"\n--- Sample Entry ---\n{text[:500]}")
        
    except Exception as e:
        print(f"Error reading CSV: {e}")
        text = "Error loading data."

In [ ]:
# --- Tokenizer ---
# Simple Character-level Tokenizer
# Note: For a "Weeb" bot, character level is fun because it can invent new Japanese-sounding names.
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocabulary Size: {vocab_size} unique characters")

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Create Train/Val Split
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# --- Hyperparameters ---
batch_size = 64      
block_size = 256     # Context length (needs to be enough to hold a full synopsis)
max_iters = 5000     # Train longer for better results
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384         
n_head = 6
n_layer = 6
dropout = 0.2

def get_batch(split):
    data_source = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_source) - block_size, (batch_size,))
    x = torch.stack([data_source[i:i+block_size] for i in ix])
    y = torch.stack([data_source[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

## 3. The Transformer Architecture
Standard GPT implementation.

In [ ]:
class Head(nn.Module):
    """ One head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ Simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            logits = logits / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

## 4. Training Loop

In [ ]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

model = GPTModel()
m = model.to(device)
print(f"Model parameters: {sum(p.numel() for p in m.parameters())/1e6:.2f} M")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

train_losses = []
val_losses = []

print("Starting training (this may take 10-15 mins on GPU)...")
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        train_losses.append(losses['train'])
        val_losses.append(losses['val'])

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Plotting the loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Evaluations')
plt.ylabel('Cross Entropy Loss')
plt.legend()
plt.show()

## 5. The Ultimate Weeb Generator
Since our model learned the format `ANIME: ...`, we can prompt it with `ANIME:` to get it to hallucinate new shows.

In [ ]:
# Set the seed context to start generation
# You can prompt it with specific genres if the dataset supports it, but usually standard completion works best.
start_str = "ANIME:"
context = torch.tensor([encode(start_str)], dtype=torch.long, device=device)

print("--- AI Generated Anime ---\n")
generated_output = decode(m.generate(context, max_new_tokens=1000, temperature=0.8).tolist()[0])
print(generated_output)

# Save model
torch.save(model.state_dict(), 'anime_gpt.pth')